In [1]:
from langchain_core.output_parsers import JsonOutputParser
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import OpenAI

class Question(BaseModel):
    Question: str = Field(description="question to check the knowledge level")
    answers: list = Field(description="list of 4 possible answers, only one is correct, include A, B, C, D before question")
    correct_answer: str = Field(description="correct answer")

class Questions(BaseModel):
    Questions: List[Question] = Field(description="question object")

# And a query intented to prompt a language model to populate the data structure.
question_query = "Your job is to determine the level of knowledge of a user. Given a topic ask 2 multiple choice questions with 4 possible answers each, that help to determine the knowledge level."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Questions)

prompt = PromptTemplate(
    template="The topic is {topic}.\n{format_instructions}\n{query}\n",
    input_variables=["query", "topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm = OpenAI(openai_api_key="KEY")

chain = prompt | llm | parser
topic = 'climate change'

chain.invoke({"query": question_query, "topic": topic})

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'Questions': [{'Question': 'What is the main cause of climate change?',
   'answers': ['A. Volcanic activity',
    'B. Natural cycles',
    'C. Human activities',
    "D. Sun's radiation"],
   'correct_answer': 'C. Human activities'},
  {'Question': 'Which of the following is not a greenhouse gas?',
   'answers': ['A. Carbon dioxide',
    'B. Water vapor',
    'C. Nitrogen',
    'D. Methane'],
   'correct_answer': 'C. Nitrogen'}]}